In [19]:
import sys
import warnings
warnings.filterwarnings("ignore")

import cv2
from keras.models import load_model
import numpy as np

from utils.datasets import get_labels
from utils.inference import detect_faces
from utils.inference import draw_text
from utils.inference import draw_bounding_box
from utils.inference import apply_offsets
from utils.inference import load_detection_model
from utils.inference import load_image
from utils.preprocessor import preprocess_input
import tensorflow as tf

In [20]:
# hyper-parameters for bounding boxes shape
gender_offsets = (30, 60)
gender_offsets = (10, 10)
emotion_offsets = (20, 40)
emotion_offsets = (0, 0)

In [21]:
detection_model_path = '../trained_models/detection_models/haarcascade_frontalface_default.xml'
emotion_model_path = '../trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5'
gender_model_path = '../trained_models/gender_models/simple_CNN.81-0.96.hdf5'

In [22]:
emotion_labels = get_labels('fer2013')
gender_labels = get_labels('imdb')

In [24]:
face_detection = load_detection_model(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)
gender_classifier = load_model(gender_model_path, compile=False)

In [25]:

# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]
gender_target_size = gender_classifier.input_shape[1:3]

In [26]:
from keras.preprocessing import image


In [8]:
def remove_transparency(im, bg_colour=(255, 255, 255)):

    # Only process if image has transparency 
    if im.mode in ('RGBA', 'LA') or (im.mode == 'P' and 'transparency' in im.info):

        # Need to convert to RGBA if LA format due to a bug in PIL 
        alpha = im.convert('RGBA').split()[-1]

        # Create a new background image of our matt color.
        # Must be RGBA because paste requires both images have the same format

        bg = Image.new("RGBA", im.size, bg_colour + (255,))
        bg.paste(im, mask=alpha)
        return bg

    else:
        return im

In [9]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
def converter(x):

    #x has shape (batch, width, height, channels)
    return (0.21 * x[:,:,:1]) + (0.72 * x[:,:,1:2]) + (0.07 * x[:,:,-1:])

In [268]:
target_size=None
from skimage import io
from skimage.color import rgb2gray
from skimage import img_as_ubyte
rgb_img = io.imread(image_path)

In [27]:
rgb_image=image.img_to_array(image.load_img(image_path, target_size=None,color_mode='rgb'))
# gray_img=rgb2gray(rgb_img)
# gray_image = img_as_ubyte(gray_img)
gray_image=converter(rgb_image)
# print(gray_image.shape)
gray_image = np.squeeze(gray_image)
gray_image = gray_image.astype('uint8')
gray_image.shape

(1024, 1024)

In [21]:
faces = detect_faces(face_detection, gray_image)
print('Length:',len(faces))
for face_coordinates in faces:
    x1, x2, y1, y2 = apply_offsets(face_coordinates, gender_offsets)
    rgb_face = rgb_image[y1:y2, x1:x2]

    x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
    gray_face = gray_image[y1:y2, x1:x2]

    try:
        rgb_face = cv2.resize(rgb_face, (gender_target_size))
        gray_face = cv2.resize(gray_face, (emotion_target_size))
    except:
        continue

    rgb_face = preprocess_input(rgb_face, False)
    rgb_face = np.expand_dims(rgb_face, 0)
    gender_prediction = gender_classifier.predict(rgb_face)
    gender_label_arg = np.argmax(gender_prediction)
    gender_text = gender_labels[gender_label_arg]

    gray_face = preprocess_input(gray_face, True)
    gray_face = np.expand_dims(gray_face, 0)
    gray_face = np.expand_dims(gray_face, -1)
    emotion_label_arg = np.argmax(emotion_classifier.predict(gray_face))
    emotion_text = emotion_labels[emotion_label_arg]
    print(emotion_label_arg)
    print(emotion_text)
    print(np.array(emotion_classifier.predict(gray_face))*100)
    if gender_text == gender_labels[0]:
        color = (0, 0, 255)
    else:
        color = (255, 0, 0)

    draw_bounding_box(face_coordinates, rgb_image, color)
    draw_text(face_coordinates, rgb_image, gender_text, color, 0, -20, 1, 2)
    draw_text(face_coordinates, rgb_image, emotion_text, color, 0, -50, 1, 2)

Length: 1
4
sad
[[ 0.772  0.555  1.267  0.125 76.294  0.045 20.943]]


In [22]:
bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
cv2.imwrite('../images/predicted_test_image.png', bgr_image)

True

In [41]:
import lime 

In [42]:
from lime import lime_image

In [43]:
explainer = lime_image.LimeImageExplainer()

In [282]:
# def gender_preprocess
# def gender_predict

gray_image=converter(rgb_img)
# print(gray_image.shape)
gray_image = np.squeeze(gray_image)
gray_image = gray_image.astype('uint8')
# print('HERE....')
faces = detect_faces(face_detection, gray_image)
print(len(faces))

1


In [308]:
def process_gray(img):
    faces = detect_faces(face_detection, img)
    print(len(faces))
#     if len(faces)>0:
    x1, x2, y1, y2 = apply_offsets(faces[0], emotion_offsets)

    gray_face = gray_image[y1:y2, x1:x2]
    gray_face = cv2.resize(gray_face, (emotion_target_size))
    return gray_face

In [26]:
def emotion_preprocess(img):
    gray_image=converter(img)
    # print(gray_image.shape)
    gray_image = np.squeeze(gray_image)
    gray_image = gray_image.astype('uint8')
    print('HERE....')
    faces = detect_faces(face_detection, img)
    print(len(faces))
#     if len(faces)>0:
    x1, x2, y1, y2 = apply_offsets(faces[0], emotion_offsets)

    gray_face = gray_image[y1:y2, x1:x2]
    gray_face = cv2.resize(gray_face, (emotion_target_size))

    gray_face = preprocess_input(gray_face, True)
    gray_face = np.expand_dims(gray_face, 0)
    gray_face = np.expand_dims(gray_face, -1)

    return gray_face
#     else:
#         raise Exception('No Face Detected')

In [26]:
import random

def emotion_predict(img):
    gray_image=converter(img)
    # print(gray_image.shape)
    gray_image = np.squeeze(gray_image)
    gray_image = gray_image.astype('uint8')
#     print('HERE....')
    faces = detect_faces(face_detection, gray_image)

    if len(faces)==0:
        rand_index=random.randint(0,6)
        ret_arr=np.zeros((1,7))
        ret_arr[rand_index]=1.0
        return ret_arr
    x1, x2, y1, y2 = apply_offsets(faces[0], emotion_offsets)

    gray_face = gray_image[y1:y2, x1:x2]
    gray_face = cv2.resize(gray_face, (emotion_target_size))

    gray_face = preprocess_input(gray_face, True)
    gray_face = np.expand_dims(gray_face, 0)
    gray_face = np.expand_dims(gray_face, -1)
    return emotion_classifier.predict(gray_face)

In [27]:
import numpy as np
emotion_predict(rgb_img)*100
# np.zeros((1,7))

NameError: name 'rgb_img' is not defined

In [60]:
np.zeros((1,7)).astype('float32')

array([[0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [61]:
# display(image.load_img(image_path, target_size=None,color_mode='rgba
# image.img_to_array(test_img)
# x = np.expand_dims(rgb_img, axis=0)
# x.shape

In [48]:
explanation = explainer.explain_instance(rgb_image, emotion_predict, top_labels=1, hide_color=0, num_samples=100,batch_size=1)

In [55]:
explanation = explainer.explain_instance(rgb_image, emotion_predict, top_labels=1, hide_color=0, num_samples=1000,batch_size=1)

6

In [6]:
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
%matplotlib inline
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mask)

NameError: name 'explanation' is not defined

In [12]:
# emotion_classifier.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 62, 62, 8)    72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 62, 62, 8)    32          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 62, 62, 8)    0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [31]:
feat_extractor = Model(inputs=emotion_classifier.input, outputs=emotion_classifier.get_layer("global_average_pooling2d_1").output)
# feat_extractor.summary()

In [32]:
def feature_predict(img):
    gray_image=converter(img)
    # print(gray_image.shape)
    gray_image = np.squeeze(gray_image)
    gray_image = gray_image.astype('uint8')
#     print('HERE....')
    faces = detect_faces(face_detection, gray_image)

    if len(faces)==0:
        return np.zeros((1,7))
    x1, x2, y1, y2 = apply_offsets(faces[0], emotion_offsets)

    gray_face = gray_image[y1:y2, x1:x2]
    gray_face = cv2.resize(gray_face, (emotion_target_size))

    gray_face = preprocess_input(gray_face, True)
    gray_face = np.expand_dims(gray_face, 0)
    gray_face = np.expand_dims(gray_face, -1)
    return feat_extractor.predict(gray_face)

In [34]:
feature_predict(rgb_image)

array([[-0.821, -1.151, -0.326, -2.645,  3.772, -3.668,  2.48 ]],
      dtype=float32)

In [38]:
import os
features=[]
for file in os.listdir('../../Dataset/gan_faces/'):
    image_path='../../Dataset/gan_faces/'+file
    rgb_image=image.img_to_array(image.load_img(image_path, target_size=None,color_mode='rgb'))
    features.append(feature_predict(rgb_image))

In [39]:
print(features[0])

[[-0.821 -1.151 -0.326 -2.645  3.772 -3.668  2.48 ]]
